In [45]:
import numpy as np
import matplotlib.pyplot as plt



# Define simulation function
def simulate_boarding():
    # Initialize Constants
    # Define number of rows and columns
    n_rows = 33
    n_cols = 6
    
    # Calculate number of passengers
    n_pass = n_rows * n_cols

    # Create seat matrix, using -1 to represent unoccupied seats
    seats = np.zeros((n_rows, n_cols))
    seats[:, :] = -1

    # Create aisle array
    aisle_q = np.zeros(n_rows)
    aisle_q[:] = -1

    # Create initial passenger number queue
    pass_q = [int(i) for i in range(n_pass)]
    pass_q = np.array(pass_q)

    # Define seating disobedience and queue disobedience constants
    seat_disobediance = 0.05  # Smaller number for seating disobedience
    queue_disobedience = 0.1  # Adjust the value as needed

    # Calculate the extent of shuffling based on queue_disobedience
    shuffle_factor = int(queue_disobedience * n_pass)
    shuffle_factor = min(shuffle_factor, n_pass)

    # Shuffle the first 'shuffle_factor' passengers in the queue
    if shuffle_factor > 0:
        np.random.shuffle(pass_q[:shuffle_factor])
    
    # Account for passenger ages and physical passenger disabilities
    # Define the parameters for the age distribution
    mean_age = 40
    stddev_age = 15

    # Generate passenger ages based on the custom distribution
    passenger_ages = np.random.normal(mean_age, stddev_age, n_pass)
    # Ensure the ages are non-negative
    passenger_ages = np.maximum(passenger_ages, 0)
    # Round the ages to whole numbers
    passenger_ages = np.round(passenger_ages)
    
    passenger_disabilities = np.random.choice([0, 1], n_pass, p=[0.95, 0.05])  # 10% chance of having a disability

    # Define group sizes based on your criteria
    group_sizes = np.random.choice([1, 2, 3, 4], n_pass, p=[0.3, 0.3, 0.2, 0.2])  # 4 least common, 3 more common, 2 more common, and 1 most common

    # Create array for seat nos
    row_q_init = np.zeros(n_pass)
    col_q_init = np.zeros(n_pass)

    # Let's create moveto arrays
    moveto_loc = np.zeros(n_pass)
    moveto_time = np.zeros(n_pass)

    moveto_loc_dict = {i: j for i in pass_q for j in moveto_loc}
    moveto_time_dict = {i: j for i in pass_q for j in moveto_time}


    def AssignSeats(rq, cq, assign_type, group_sizes):
        n_pass = len(rq)  # Number of passengers
        n_rows = 33  # Number of rows
        n_cols = 6   # Number of columns

        if assign_type == "SINP":
            # Initialize initial and final positions
            i = 0
            f = n_rows

            # Define column seating positions 
            c = [0, 5, 1, 4, 2, 3]

            # Define iteration counter
            count = 0

            # Assign queue
            while f <= n_pass:
                rq[i:f] = list(reversed(range(0, n_rows)))
                cq[i:f] = [c[count]] * n_rows
                i += n_rows
                f += n_rows
                count += 1

        elif assign_type == "Random":
            # Create a list to hold all the seat positions
            all_seats = []

            for group_size in group_sizes:
                # Initialize possible row positions, 0 to 32
                av_rows = np.arange(0, n_rows, 1)
                # Make as many copies of these positions as the number of columns
                av_rows = np.tile(av_rows, (n_cols, 1))
                av_rows = av_rows.T.flatten()

                # Initialize possible column positions
                av_cols = np.arange(0, n_cols, 1)
                # Make as many copies of these positions as the number of rows
                av_cols = np.tile(av_cols, (n_rows, 1)).flatten()

                # Create list of all possible seat positions for this group
                group_seats = np.column_stack((av_rows, av_cols))
                np.random.shuffle(group_seats)

                # Take a subset of the group seats to accommodate the group size
                group_seats = group_seats[:group_size]

                # Append the group seats to the list of all seats
                all_seats.extend(group_seats)

            np.random.shuffle(all_seats)
            rq = np.array(all_seats)[:, 0]
            cq = np.array(all_seats)[:, 1]

        elif assign_type == "BTF":
            # Assign seats in a Back-to-Front manner
            av_rows = np.arange(0, n_rows, 1)
            av_rows = np.tile(av_rows, (n_cols, 1))
            av_rows = av_rows.T.flatten()
            av_cols = np.arange(0, n_cols, 1)
            av_cols = np.tile(av_cols, (n_rows, 1)).flatten()
            av_seats = np.column_stack((av_rows, av_cols))
            np.random.shuffle(av_seats)
            # Assign seats to passengers in groups
            group_start = 0
            for group_size in group_sizes:
                group_seats = av_seats[group_start:group_start + group_size]
                rq[group_start:group_start + group_size] = group_seats[:, 0]
                cq[group_start:group_start + group_size] = group_seats[:, 1]
                group_start += group_size

        elif assign_type == "FTB":
            # Assign seats in a Front-to-Back manner
            av_rows = np.arange(0, n_rows, 1)
            av_rows = np.tile(av_rows, (n_cols, 1))
            av_rows = av_rows.T.flatten()
            av_cols = np.arange(0, n_cols, 1)
            av_cols = np.tile(av_cols, (n_rows, 1)).flatten()
            av_seats = np.column_stack((av_rows, av_cols))
            np.random.shuffle(av_seats)
            # Assign seats to passengers in groups
            group_start = 0
            for group_size in group_sizes:
                group_seats = av_seats[group_start:group_start + group_size]
                rq[group_start:group_start + group_size] = group_seats[:, 0]
                cq[group_start:group_start + group_size] = group_seats[:, 1]
                group_start += group_size

        elif assign_type == "WMA":
            window_1 = np.array([0] * n_rows)
            rows_1 = np.arange(0, n_rows, 1)
            window_2 = np.array([5] * n_rows)
            rows_2 = np.arange(0, n_rows, 1)
            window = np.concatenate((window_1, window_2))
            rows = np.concatenate((rows_1, rows_2))
            av_seats_w = np.column_stack((rows, window))
            np.random.shuffle(av_seats_w)

            middle_1 = np.array([1] * n_rows)
            middle_2 = np.array([4] * n_rows)
            middle = np.concatenate((middle_1, middle_2))
            av_seats_m = np.column_stack((rows, middle))
            np.random.shuffle(av_seats_m)

            aisle_1 = np.array([2] * n_rows)
            aisle_2 = np.array([3] * n_rows)
            aisle = np.concatenate((aisle_1, aisle_2))
            av_seats_a = np.column_stack((rows, aisle))
            np.random.shuffle(av_seats_a)

            av_seats = np.concatenate((av_seats_w, av_seats_m, av_seats_a))
            rq = av_seats[:, 0]
            cq = av_seats[:, 1]

        elif assign_type == "Southwest":
            # Make an array [0, 5, 0, 5, ...]
            window = np.array([0, 5] * n_rows)

            # Make an array [0, 0, 1, 1, 2, 2, ...]
            rows_1 = np.arange(0, n_rows, 1)
            rows_2 = np.arange(0, n_rows, 1)
            rows = np.ravel(np.column_stack((rows_1, rows_2)))

            w_seats = np.column_stack((rows, window))
            w_group1 = w_seats[:32, :]
            w_group2 = w_seats[32:, :]

            aisle = np.array([2, 3] * n_rows)
            a_seats = np.column_stack((rows, aisle))
            a_group1 = a_seats[:32, :]
            a_group2 = a_seats[32:, :]

            mega_group1 = np.concatenate((w_group1, a_group1))
            np.random.shuffle(mega_group1)
            mega_group2 = np.concatenate((w_group2, a_group2))
            np.random.shuffle(mega_group2)

            w_and_a = np.concatenate((mega_group1, mega_group2))

            middle = np.array([1, 4] * n_rows)
            m_seats = np.column_stack((rows, middle))
            m_group1 = m_seats[:32, :]
            np.random.shuffle(m_group1)
            m_group2 = m_seats[32:, :]
            np.random.shuffle(m_group2)

            av_seats = np.concatenate((w_and_a, m_group1, m_group2))
            rq = av_seats[:, 0]
            cq = av_seats[:, 1]

        return rq, cq



    ####################################################

    #Create function to move passengers into the aircraft
    def MoveToAisle(t, aisle_q, pass_q, sum_time):
        if t > sum_time[0]:
            if aisle_q[0] == -1:
                aisle_q[0] = pass_q[0].copy()
                pass_q = np.delete(pass_q, 0)
                sum_time = np.delete(sum_time, 0)
        return aisle_q, pass_q, sum_time


    ###################################################
    
    def get_walk_speed(age, disability):
        if age > 70 or age < 8:
            if disability == 1:
                mean_time = 1.2  # 20% increase for age > 70 or < 8 and disability
            else:
                mean_time = 1.2  # 20% increase for age > 70 or < 8 and no disability
        elif disability == 1:
            mean_time = 1.5  # 50% increase for passengers with disabilities
        else:
            mean_time = 1.0  # Default mean_time for other passengers

        stddev_time = 0.2
        walk_speed = np.random.normal(loc=mean_time, scale=stddev_time)
        return walk_speed
    
    # Create an array of walking speeds based on age and disability
    walk_speeds = [get_walk_speed(age, disability) for age, disability in zip(passenger_ages, passenger_disabilities)]


    ##################################################

    # Assign seating order
    group_sizes = np.random.choice([1, 2, 3, 4], n_pass, p=[0.6, 0.2, 0.1, 0.1])
    row_q, col_q = AssignSeats(row_q_init, col_q_init, "Random", group_sizes)

    # Create array for times
    mean_time = 1.
    stddev_time = 0.2
    # Random numbers from a normal distribution with mean = mean_time and standard deviation stddev_time
    time_q = walk_speeds

    # Define multipliers (+2 for stowing luggage)
    # Empty row
    empty_mult = 1 + 2
    # Aisle occupied
    aisle_mult = 4 + 2
    # Middle occupied
    middle_mult = 5 + 2
    # Aisle and middle occupied
    aisle_middle_mult = 7 + 2

    # Create seat and speed dictionary
    pass_dict = {}
    time_dict = {}

    seat_nos = np.column_stack((row_q, col_q))
    for i in range(n_pass):
        pass_dict[i] = seat_nos[i]

    for i in range(n_pass):
        time_dict[i] = time_q[i]

    # Create sum time array
    sum_time = np.zeros(n_pass)
    for i in range(n_pass):
        sum_time[i] = sum(time_q[:i+1])

    # Define initial conditions
    time = 0
    time_step = 0.1
    exit_sum = np.sum(pass_q)
    pass_sum = np.sum(seats)
    print('initial passenger sum: ', pass_sum)
    
    
    while pass_sum != exit_sum:
        # Try to move passengers inside the plane if passengers are left
        if pass_q.size != 0:
            aisle_q, pass_q, sum_time = MoveToAisle(time, aisle_q, pass_q, sum_time)

        # Scan the aisle first for non-negative units (passengers)
        for passg in aisle_q:
            if passg != -1:
                # Store the row of the passenger in the aisle
                row = int(np.where(aisle_q == passg)[0][0])
                # See if move has been assigned to the passenger in the aisle
                if moveto_time_dict[passg] != 0:
                    # If move has been assigned, check if it is time to move
                    if time > moveto_time_dict[passg]:
                        # If it is time to move, follow the procedure below
                        # Check if move is forward in the aisle or to the seat
                        if moveto_loc_dict[passg] == "a":
                            # If move is in the aisle, check if the position ahead is empty
                            if aisle_q[row + 1] == -1:
                                # If the position is empty, move the passenger ahead and free the position behind
                                aisle_q[row + 1] = passg
                                aisle_q[row] = -1
                                # Set moves to 0 again
                                moveto_loc_dict[passg] = 0
                                moveto_time_dict[passg] = 0
                        elif moveto_loc_dict[passg] == "s":
                            # If the move is to the seat,
                            # Find the seat row and column of the passenger
                            passg_row = int(pass_dict[passg][0])
                            passg_col = int(pass_dict[passg][1])
                            # Set the seat matrix position to the passenger number
                            seats[passg_row, passg_col] = passg
                            # Free the aisle
                            aisle_q[row] = -1
                elif moveto_time_dict[passg] == 0:
                    # If the move hasn't been assigned to the passenger
                    # Check the passenger's seat location
                    passg_row = int(pass_dict[passg][0])
                    passg_col = int(pass_dict[passg][1])

                    # Check if the passenger is in a group
                    if passg in group_passengers:
                        group_indices = [pass_dict[p] for p in group_passengers if p != passg]

                        # Group passengers should sit together
                        if group_indices:
                            if (passg_row, passg_col) == group_indices[0]:
                                # The passenger is at the group's seat location, designate the move type as seat
                                moveto_loc_dict[passg] = "s"
                                # Check what type of seat: aisle, middle, or window
                                # Depending upon the seat type, designate when it is time to move
                                if passg_col == 0:
                                    # Check availability of adjacent seats for the group
                                    adjacent_seats_occupied = any(seats[passg_row, 1:3] != -1)
                                    if adjacent_seats_occupied:
                                        moveto_time_dict[passg] = time + empty_mult * time_dict[passg]
                                    else:
                                        moveto_time_dict[passg] = time + aisle_middle_mult * time_dict[passg]
                                elif passg_col == 5:
                                    # Check availability of adjacent seats for the group
                                    adjacent_seats_occupied = any(seats[passg_row, 3:5] != -1)
                                    if adjacent_seats_occupied:
                                        moveto_time_dict[passg] = time + empty_mult * time_dict[passg]
                                    else:
                                        moveto_time_dict[passg] = time + aisle_middle_mult * time_dict[passg]
                                elif passg_col in {1, 4}:
                                    # Check availability of adjacent seats for the group
                                    adjacent_seat_occupied = seats[passg_row, 3] != -1
                                    if adjacent_seat_occupied:
                                        moveto_time_dict[passg] = time + empty_mult * time_dict[passg]
                                    else:
                                        moveto_time_dict[passg] = time + aisle_mult * time_dict[passg]
                                elif passg_col in {2, 3}:
                                    moveto_time_dict[passg] = time + aisle_mult * time_dict[passg]
                            else:
                                # The passenger is not at the group's seat location, designate the movement type as aisle
                                moveto_loc_dict[passg] = "a"
                                # Designate the time to move
                                moveto_time_dict[passg] = time + time_dict[passg]
                        else:
                            # No group indices found, the passenger is not in a group
                            # Handle as a single passenger (same code as before)
                            # Designate the movement type as aisle
                            moveto_loc_dict[passg] = "a"
                            # Designate the time to move
                            moveto_time_dict[passg] = time + time_dict[passg]

        # Iteration timekeeping
        time += time_step
        pass_sum = np.sum(seats)

    
    # Debugging output
    print("Time: {:.2f}, pass_sum: {}, exit_sum: {}".format(time, pass_sum, exit_sum))

    return time

        


In [46]:
# Number of simulations
num_simulations = 1

boarding_times = []

for _ in range(num_simulations):
    result = simulate_boarding()
    if result > 0:
        boarding_times.append(result)
    else:
        print("negative number!")

# Calculate the average time
average_time = sum(boarding_times) / len(boarding_times)

# Calculate percentiles
percentile_5 = np.percentile(boarding_times, 5)
percentile_95 = np.percentile(boarding_times, 95)

print("Average boarding time for {} simulations: {:.2f}".format(len(boarding_times), average_time))
print("5th percentile boarding time: {:.2f}".format(percentile_5))
print("95th percentile boarding time: {:.2f}".format(percentile_95))

# Create a histogram of boarding times
plt.hist(boarding_times, bins=20, color='blue', alpha=0.7)
plt.xlabel("Boarding Time")
plt.ylabel("Frequency")
plt.title("Histogram of Boarding Times")
plt.grid(True)
plt.show()

initial passenger sum:  -198.0


NameError: name 'group_passengers' is not defined

In [ ]:
while pass_sum != exit_sum:
        if pass_q.size != 0:
            aisle_q, pass_q, sum_time = MoveToAisle(time, aisle_q, pass_q, sum_time)

        # Initialize a list to keep track of disobedient passengers
        disobedient_passengers = []

        for passg in aisle_q:
            if passg != -1:
                row = int(np.where(aisle_q == passg)[0][0])

                # Handle disobedient passengers
                if passg in disobedient_passengers:
                    # Find the correct seat of the disobedient passenger
                    correct_passg_row = int(pass_dict[passg][0])
                    correct_passg_col = int(pass_dict[passg][1])

                    # If the passenger in this seat is not the disobedient passenger, swap them
                    if seats[correct_passg_row, correct_passg_col] != passg:
                        other_passenger = seats[correct_passg_row, correct_passg_col]

                        # Move the disobedient passenger to their correct seat
                        aisle_q[row] = -1
                        aisle_q[int(np.where(aisle_q == other_passenger)[0][0])] = passg
                        seats[correct_passg_row, correct_passg_col] = passg

                        # Move the other passenger to the disobedient passenger's previous seat
                        disobedient_passenger_row = int(pass_dict[other_passenger][0])
                        disobedient_passenger_col = int(pass_dict[other_passenger][1])
                        seats[disobedient_passenger_row, disobedient_passenger_col] = other_passenger

                        # Update the disobedient passenger's location
                        pass_dict[passg] = (correct_passg_row, correct_passg_col)

                        # Update the list of disobedient passengers
                        disobedient_passengers.remove(passg)

                # Check if move has been assigned to the passenger in the aisle
                if moveto_time_dict[passg] != 0:
                    # If move has been assigned, check if it is time to move
                    if time > moveto_time_dict[passg]:
                        # If it is time to move, follow the procedure below
                        # Check if move is forward in the aisle or to the seat
                        if moveto_loc_dict[passg] == "a":
                            # If move is in the aisle, check if the position ahead is empty
                            if aisle_q[row + 1] == -1:
                                # If the position is empty, move the passenger ahead and free the position behind
                                aisle_q[row + 1] = passg
                                aisle_q[row] = -1
                                # Set moves to 0 again
                                moveto_loc_dict[passg] = 0
                                moveto_time_dict[passg] = 0
                        elif moveto_loc_dict[passg] == "s":
                            # If the move is to the seat,
                            # Find the seat row and column of the passenger
                            passg_row = int(pass_dict[passg][0])
                            passg_col = int(pass_dict[passg][1])

                            # Check for disobedience
                            rand_num = np.random.rand()
                            if rand_num <= seat_disobediance:
                                # Assign the passenger to a random seat that is not their own
                                # Assume there are 6 columns in each row
                                wrong_col = np.random.randint(0, 6)
                                # Make sure the wrong column is not the same as the correct column
                                while wrong_col == passg_col:
                                    wrong_col = np.random.randint(0, 6)
                                # Move the disobedient passenger
                                seats[passg_row, wrong_col] = passg
                                disobedient_passengers.append(passg)
                            else:
                                # Assign the passenger to their correct seat
                                seats[passg_row, passg_col] = passg
                            aisle_q[row] = -1
                elif moveto_time_dict[passg] == 0:
                    passg_row = int(pass_dict[passg][0])
                    passg_col = int(pass_dict[passg][1])
                    if passg_row == row:
                        # If the passenger is at the row where his/her seat is
                        # Designate the move type as seat
                        moveto_loc_dict[passg] = "s"

                        # Check what type of seat: aisle, middle, or window
                        # Depending upon the seat type, designate when it is time to move
                        if passg_col == 0:
                            if seats[passg_row, 1] != -1 and seats[passg_row, 2] != -1:
                                moveto_time_dict[passg] = time + aisle_middle_mult * time_dict[passg]
                            elif seats[passg_row, 1] != -1:
                                moveto_time_dict[passg] = time + middle_mult * time_dict[passg]
                            elif seats[passg_row, 2] != -1:
                                moveto_time_dict[passg] = time + aisle_mult * time_dict[passg]
                            else:
                                moveto_time_dict[passg] = time + empty_mult * time_dict[passg]
                        # ... (similar checks for other columns)
                    elif passg_row != row:
                        # If the passenger is not at the row where his/her seat is
                        # Designate the movement type as aisle
                        moveto_loc_dict[passg] = "a"
                        # Designate the time to move
                        moveto_time_dict[passg] = time + time_dict[passg]

        # Iteration timekeeping
        time += time_step
        pass_sum = np.sum(seats)
